<a href="https://colab.research.google.com/github/bartlomiejandruchow/AI/blob/main/KerasFirstExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras import ops
import math
from keras.datasets import mnist
from keras import optimizers
import tensorflow as tf

class NaiveDense:
  def __init__(self, input_size, output_size, activation=None):
    self.activation = activation
    self.W = keras.Variable(shape=(input_size, output_size), initializer="uniform")
    self.b = keras.Variable(shape=(output_size,), initializer="zeros")

  def __call__(self,inputs):
    x = ops.matmul(inputs, self.W)
    x = x + self.b
    if self.activation is not None:
      x = self.activation(x)
    return x

  @property
  def weights(self):
    return [self.W, self.b]

class NaiveSequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, inputs):
    x = inputs
    for layer in self.layers:
      x = layer(x)
    return x

  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights += layer.weights
    return weights

model = NaiveSequential(
    [
        NaiveDense(input_size=28*28, output_size=512, activation=ops.relu),
        NaiveDense(input_size=512, output_size=10, activation=ops.softmax)
    ]
)
assert len(model.weights) == 4

class BatchGenerator:
  def __init__(self, images, labels, batch_size=128):
    assert len(images) == len(labels)
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images) / batch_size)

  def next(self):
    images = self.images[self.index:self.index+self.batch_size]
    labels = self.labels[self.index:self.index+self.batch_size]
    self.index += self.batch_size
    return images, labels



learning_rate = 1e-3

def update_weights(gradients, weights):
  for g, w in zip(gradients, weights):
    w.assign(w-g*learning_rate)


def one_training_step(model, images_batch, labels_batch):
  with tf.GradientTape() as tape:
    predictions = model(images_batch)
    loss = ops.sparse_categorical_crossentropy(labels_batch, predictions)
    avarage_loss = ops.mean(loss)
  gradients = tape.gradient(avarage_loss, model.weights)
  update_weights(gradients, model.weights)
  return avarage_loss

def fit(model, images, labels, epochs, batch_size=128):
  for epoch_counter in range(epochs):
    print(f"Epoch {epoch_counter}")
    batch_generator = BatchGenerator(images, labels)
    for batch_counter in range(batch_generator.num_batches):
      images_batch, labels_batch = batch_generator.next()
      loss = one_training_step(model, images_batch, labels_batch)
      if batch_counter % 100 == 0:
        print(f'loss at batch {batch_counter} : {loss:.2f}')


In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000,28*28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000,28*28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size = 128)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 0
loss at batch 0 : 2.31
loss at batch 100 : 2.29
loss at batch 200 : 2.27
loss at batch 300 : 2.22
loss at batch 400 : 2.20
Epoch 1
loss at batch 0 : 2.14
loss at batch 100 : 2.15
loss at batch 200 : 2.11
loss at batch 300 : 2.06
loss at batch 400 : 2.04
Epoch 2
loss at batch 0 : 1.97
loss at batch 100 : 2.00
loss at batch 200 : 1.93
loss at batch 300 : 1.90
loss at batch 400 : 1.87
Epoch 3
loss at batch 0 : 1.79
loss at batch 100 : 1.83
loss at batch 200 : 1.74
loss at batch 300 : 1.71
loss at batch 400 : 1.69
Epoch 4
loss at batch 0 : 1.59
loss at batch 100 : 1.65
loss at batch 200 : 1.55
loss at batch 300 : 1.52
loss at batch 400 : 1.52
Epoch 5
loss at batch 0 : 1.40
loss at batch 100 : 1.48
loss at batch 200 : 1.36
loss at batch 300 : 1.34
loss at batch 400 : 1.36
Epoch 6
loss at batch 0 : 1.24
loss at batch 100 : 1.32
loss at batch 200 : 1.19
loss at batch 300 : 1.19
loss at batch 400 : 1.22
Epoch 7
loss at batch 0 : 1.09
l

In [7]:
predictions = model(test_images)
prediction_labels = ops.argmax(predictions, axis=1)
matches = prediction_labels == test_labels
print(f"accuracy: {ops.mean(matches):.2f}")



accuracy: 0.84
